In [1]:
import os
import numpy as np
import torch
from PIL import Image
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

import gc
import numpy as np
import pandas as pd
import torch
import os
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import ConcatDataset, DataLoader, Subset, Dataset
from torchvision.datasets import DatasetFolder, VisionDataset
# This is for the progress bar.
from tqdm.auto import tqdm
import random
from pathlib import Path
import math
import cv2


In [2]:
! pip install --no-index --find-links /kaggle/input/pycocotools/wheelhouse pycocotools

Looking in links: /kaggle/input/pycocotools/wheelhouse
Processing /kaggle/input/pycocotools/wheelhouse/pycocotools-2.0.6-cp310-cp310-linux_x86_64.whl


In [3]:
import base64
import numpy as np
from pycocotools import _mask as coco_mask
import typing as t
import zlib


def encode_binary_mask(mask: np.ndarray) -> t.Text:
    """Converts a binary mask into OID challenge encoding ascii text."""

    # check input mask --
    if mask.dtype != np.bool_:
        raise ValueError(
            "encode_binary_mask expects a binary mask, received dtype == %s" %
            mask.dtype)

    mask = np.squeeze(mask)
    if len(mask.shape) != 2:
        raise ValueError(
            "encode_binary_mask expects a 2d mask, received shape == %s" %
            mask.shape)

  # convert input mask to expected COCO API input --
    mask_to_encode = mask.reshape(mask.shape[0], mask.shape[1], 1)
    mask_to_encode = mask_to_encode.astype(np.uint8)
    mask_to_encode = np.asfortranarray(mask_to_encode)

    # RLE encode mask --
    encoded_mask = coco_mask.encode(mask_to_encode)[0]["counts"]

    # compress and base64 encoding --
    binary_str = zlib.compress(encoded_mask, zlib.Z_BEST_COMPRESSION)
    base64_str = base64.b64encode(binary_str)
    return base64_str

In [4]:
def get_model_instance_segmentation(num_classes):
    
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(weights = None , weights_backbone = None)
    
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)
    
    return model

In [5]:
class KidneyTestingDataset(Dataset):
    def __init__(self , image_list , tfm):
        self.image_list = image_list
        self.tfm = tfm
        

    def __len__(self):
        return len(self.image_list)
    
    def __getitem__(self,idx):
        
        img = Image.open(self.image_list[idx])
        img = self.tfm(img)
        return self.image_list[idx] , img
        

In [6]:
ROOT = Path("/kaggle/input/hubmap-hacking-the-human-vasculature")

image_folder = "test"

image_list = [str(i) for i  in (ROOT / image_folder).glob('*.tif')]
tfm = transforms.Compose([
    transforms.ToTensor(),
]) 


In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# Initialize a model, and put it on the device specified.
model = get_model_instance_segmentation(2).to(device)
model.load_state_dict(torch.load("/kaggle/input/hubmap-model/mask-rcnn-model_best.ckpt"))
batch_size = 4
threshold = 0.3
model.eval()
pass



In [8]:
# testset = KidneyTestingDataset(image_list , tfm)
# test_loader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)

df = pd.DataFrame(columns=['id' ,'height' ,'width', 'prediction_string'])


for i in (ROOT / image_folder).glob('*.tif'):
    img = Image.open(i)
    img = tfm(img)
    rel = model(img.unsqueeze(0).to(device))[0]

    masks = rel["masks"].to("cpu").detach().numpy().astype(bool)
    boxes = rel["boxes"].to("cpu").detach().numpy()
    score = rel["scores"].to("cpu").detach().numpy()

    pred_string = ""
    
    for idx in range(len(masks)):
#         print(masks[idx][0].shape)
        if score[idx] > 0.5:
            binary_mask = encode_binary_mask(masks[idx][0]).decode("utf-8")
            pred_string += f"0 {score[idx]} {binary_mask} "
            
#     print(pred_string)
#     if pred_string == "":
#         pred_string = "0 1.0 eNoLTDAwyrM3yI/PMwcAE94DZA=="
    
    
    new_line = pd.Series({'id': i.stem, 'height': 512 , 'width': 512 , 'prediction_string':pred_string})
    df = pd.concat([df, new_line.to_frame().T], ignore_index=True)
    

    
df.to_csv("submission.csv" , index=False)
print(df )

             id height width  \
0  72e40acccadf    512   512   

                                   prediction_string  
0  0 0.6006835103034973 eNoLNwwxzLEzoC7IjIs2BwAkP...  
